In [16]:
# This is an experiment notebook.
# Finalized functions are in utilities/visualization_utilities.py

%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from joblib import Parallel, delayed
import time

from skimage.transform import rescale

In [17]:
def scoremap_overlay(stack, sec, name, downscale_factor, image_shape=None, return_mask=True):
    '''
    Generate scoremap of structure.
    name: name
    image_shape: (w, h)
    '''
    
    if image_shape is None:
        w, h = DataManager(stack=stack).get_image_dimension()
    else:
        w, h = image_shape
            
    scoremap_bp_filepath = scoremaps_rootdir + '/%(stack)s/%(slice)04d/%(stack)s_%(slice)04d_roi1_denseScoreMapLossless_%(label)s.hdf' \
        % {'stack': stack, 'slice': sec, 'label': name}

    if not os.path.exists(scoremap_bp_filepath):
        sys.stderr.write('No scoremap for %s for section %d\n' % (name, sec))
        return

    scoremap = load_hdf(scoremap_bp_filepath)
    
    interpolation_xmin, interpolation_xmax, \
    interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_rootdir, 
                 '%(stack)s/%(sec)04d/%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                                     {'stack': stack, 'sec': sec, 'label': name})).astype(np.int)

    dense_score_map_lossless = np.zeros((h, w))
    dense_score_map_lossless[interpolation_ymin:interpolation_ymax+1,
                             interpolation_xmin:interpolation_xmax+1] = scoremap
    
    mask = dense_score_map_lossless > 0.
    
    scoremap_viz = plt.cm.hot(dense_score_map_lossless[::downscale_factor, ::downscale_factor])[..., :3]
    viz = img_as_ubyte(scoremap_viz)
    
    if return_mask:
        return viz, mask
    else:
        return viz

In [18]:
def scoremap_overlay_on(bg, stack, sec, name, downscale_factor, export_filepath=None, label_text=True):
    
    if bg == 'original':
        p = DataManager.get_image_filepath(stack=stack, section=sec, version='rgb-jpg')
        bg = imread(p)

    ret = scoremap_overlay(stack, sec, name, downscale_factor, image_shape=bg.shape[:2])
    
    if ret is None:
        return None
    
    scoremap_viz, mask = ret
    
    m = mask[::downscale_factor, ::downscale_factor]
    
    viz = bg[::downscale_factor, ::downscale_factor].copy()
    viz[m] = (.3 * img_as_ubyte(scoremap_viz[m, :3]) + .7 * viz[m]).astype(np.uint8)
    
    # put label name at left upper corner
    if label_text:
        cv2.putText(viz, name, (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)

    # export image to disk
    if export_filepath is not None:
        create_if_not_exists(os.path.dirname(export_filepath))
        cv2.imwrite(export_filepath, img_as_ubyte(viz[..., [2,1,0]]))
        
    return viz

In [19]:
def export_scoremaps_worker(bg, stack, sec, names, downscale_factor, export_filepath_fmt=None, label_text=True):
    
    if bg == 'original':
        img = imread(DataManager.get_image_filepath(stack=stack, section=sec, version='rgb-jpg'))

    for name in names:
        export_filepath = export_filepath_fmt % {'stack': stack, 'sec': sec, 'name': name}
        scoremap_overlay_on(img, stack, sec, name, downscale_factor, export_filepath, label_text)

In [20]:
def export_scoremaps(bg, stack, sections, names, downscale_factor, export_filepath_fmt=None, label_text=True):
    
    t = time.time()
    
    Parallel(n_jobs=8)(delayed(export_scoremaps_worker)(bg, stack, sec, names, downscale_factor, 
                                                        export_filepath_fmt=export_filepath_fmt, 
                                                        label_text=label_text) 
                       for sec in sections)
    
    print time.time() - t
    
#     for sec in sections:
#         if bg == 'original':
#             img = imread(DataManager.get_image_filepath(stack=stack, section=sec, version='rgb-jpg'))

#         for name in names:
#             export_filepath = export_filepath_fmt % {'stack': stack, 'sec': sec, 'name': name}
#             scoremap_overlay_on(img, stack, sec, name, downscale_factor, export_filepath, label_text)

In [70]:
t = time.time()
viz = scoremap_overlay_on('original', stack, 156, '7N', 8)
print time.time() - t
display_image(viz)

4.75361585617


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [71]:
display_image(viz)

/oasis/projects/nsf/csd395/yuncong/Brain/vis/tmp.jpg

In [ ]:
# for stack in all_stacks:
for stack in ['MD603']:

    first_sec, last_sec = section_range_lookup[stack]

    export_filepath_fmt = scoremapViz_rootdir + '/%(name)s/%(stack)s/%(stack)s_%(sec)04d_roi1_scoremapViz_%(name)s.jpg'
    export_scoremaps('original', stack, range(first_sec, last_sec+1), ['7N', '5N'], 8, 
                     export_filepath_fmt=export_filepath_fmt, label_text=True)

In [21]:
def get_labeling_list():
    
    from collections import defaultdict
    import pandas as pd

    labeling_list = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    for stack in os.listdir(os.environ['LABELING_DIR']):
        if os.path.isdir(os.path.join(os.environ['LABELING_DIR'], stack)):
            for sec in sorted(os.listdir(os.path.join(os.environ['LABELING_DIR'], stack))):
                for labeling_fn in os.listdir(os.path.join(os.environ['LABELING_DIR'], stack, sec)):
                    user = labeling_fn.split('_')[2]
                    labeling_list[stack][int(sec)][user].append(labeling_fn)

#     labeling_list.default_factory = None
    
    reformed = {(stack, sec): labeling_list[stack][sec] for stack, secs in labeling_list.iteritems() for sec in secs }
    df = pd.DataFrame(reformed)
    
    return df

In [22]:
labeling_list = get_labeling_list()

In [23]:
def load_annotation(stack, section, username=None, timestamp='latest', path_only=False):

    import cPickle as pickle
    from itertools import chain
    
    try:
        if timestamp == 'latest':
            if username is None:
                annotation_names = list(chain.from_iterable(labeling_list[stack][section].dropna().__iter__()))
                annotation_name = sorted(annotation_names)[-1]
            else:
                annotation_name = sorted(labeling_list[stack][section][username])[-1]

            _, _, _, timestamp, _ = annotation_name.split('_')
        else:
            sys.stderr.write('Timestamp is not latest, not implemented.\n')
            return
            
    except Exception as e:
        sys.stderr.write('Annotation does not exist: %s, %d, %s, %s\n' % (stack, section, username, timestamp))
        return
        
    annotation_filepath = os.path.join(os.environ['LABELING_DIR'], stack, '%04d'%section, annotation_name)
    
    if path_only:
        return annotation_filepath
    else:        
        return pickle.load(open(annotation_filepath, 'r')), timestamp

In [24]:
def annotation_overlay_on(bg, stack, section, structure_names=None, downscale_factor=8, 
                          users=['yuncong', 'autoAnnotate'], colors=None, show_labels=True,
                         export_filepath_fmt=None):
        
    annotations = {}
    timestamps = {}
    
    if colors is None:
        colors = [(0,0,255), (255,0,0), (0,255,0)]
    
    for user in users:
        ret = load_annotation(stack=stack, section=section, username=user)
        if ret is not None:
            annotations[user] = ret[0]
            timestamps[user] = ret[1]
    
    if len(annotations) == 0:
        return
        
    if bg == 'original':
        bg = imread(DataManager.get_image_filepath(stack=stack, section=section, version='rgb-jpg'))
        viz = bg[::downscale_factor, ::downscale_factor].copy()
    else:
        viz = bg.copy()

    added_labels = set([])
            
    for user, anns in annotations.iteritems():
        for ann in anns:
            
            if structure_names is not None and ann['label'] not in structure_names:
                continue
            
#             vertices_ref = np.array(ann['vertices']) / downscale_factor
            
            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (255,0,0), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, colors[users.index(user)], 2)
            
            if show_labels:
            
                if ann['label'] not in added_labels:            
                    lx, ly = np.array(ann['labelPos']) / downscale_factor
                    cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                                cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)
                    
                    added_labels.add(ann['label'])
    
    
    if export_filepath_fmt is not None:
        if len(structure_names) == 1:
            export_filepath = export_filepath_fmt % {'stack': stack, 'sec': section, 'name': structure_names[0],
                                                 'annofn': '_'.join(usr+'_'+ts for usr, ts in timestamps.iteritems())}
        else:
            export_filepath = export_filepath_fmt % {'stack': stack, 'sec': section, 
                                                 'annofn': '_'.join(usr+'_'+ts for usr, ts in timestamps.iteritems())}
                        
        create_if_not_exists(os.path.dirname(export_filepath))
        cv2.imwrite(export_filepath, viz[..., ::-1])

#     if ret is not None and ret_autoAnno is not None:
#         output_fn = os.path.join(labelingViz_dir, 
#                      '%(stack)s_%(sec)04d_%(user)s_%(ts)s_%(user2)s_%(ts2)s_viz.jpg'% \
#                                  {'stack':stack, 'sec':sec, 'user': user, 'ts': ts, 'user2':user2, 'ts2':ts2})
#     elif ret is not None:
#         output_fn = os.path.join(labelingViz_dir, 
#                      '%(stack)s_%(sec)04d_%(user)s_%(ts)s_viz.jpg'% \
#                                  {'stack':stack, 'sec':sec, 'user': user, 'ts': ts})            
#     else:
#         output_fn = os.path.join(labelingViz_dir, 
#              '%(stack)s_%(sec)04d_%(user2)s_%(ts2)s_viz.jpg'% \
#                          {'stack':stack, 'sec':sec, 'user2': user2, 'ts2': ts2})
            
#     cv2.imwrite(output_fn, viz[..., ::-1])
    
    return viz

In [25]:
outputViz_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_scoremapPlusAnnotationViz'

In [ ]:
# stack = 'MD594'
# sec = 156

# for name in set(labels_unsided) - {'outerContour'}:
#     scoremap_overlaid = scoremap_overlay_on('original', stack, sec, name, 8)
    
#     export_filepath_fmt = outputViz_rootdir + '/%(name)s/%(stack)s/%(stack)s_%(sec)04d_roi1_scoremapPlusAnnotationViz_%(name)s_%(annofn)s.jpg'
    
#     annotation_overlaid = annotation_overlay_on(scoremap_overlaid, stack, sec, [name], 8,
#                                                export_filepath_fmt=export_filepath_fmt)
    

In [26]:
def export_scoremapPlusAnnotationVizs_worker(bg, stack, sec, names, downscale_factor, export_filepath_fmt=None):
    
    if bg == 'original':
        img = imread(DataManager.get_image_filepath(stack=stack, section=sec, version='rgb-jpg'))

    for name in names:
        scoremap_overlaid = scoremap_overlay_on(img, stack, sec, name, downscale_factor, label_text=True)
        annotation_overlaid = annotation_overlay_on(scoremap_overlaid, stack, sec, [name], 8,
                                               export_filepath_fmt=export_filepath_fmt)

In [27]:
def export_scoremapPlusAnnotationVizs(bg, stack, sections, names, downscale_factor, export_filepath_fmt=None):
    
    Parallel(n_jobs=4)(delayed(export_scoremapPlusAnnotationVizs_worker)(bg, stack, sec, names, downscale_factor, 
                                                        export_filepath_fmt=export_filepath_fmt) 
                       for sec in sections)
    
#     for sec in sections:
#         export_scoremapPlusAnnotationVizs_worker(bg, stack, sec, names, downscale_factor, 
#                                                         export_filepath_fmt=export_filepath_fmt)

In [15]:
export_filepath_fmt = outputViz_rootdir + '/%(name)s/%(stack)s/%(stack)s_%(sec)04d_roi1_scoremapPlusAnnotationViz_%(name)s_%(annofn)s.jpg'

t = time.time()

stack = 'MD585'
sec = 160
export_scoremapPlusAnnotationVizs('original', stack, range(sec, sec+12), ['7N'], 8, export_filepath_fmt=export_filepath_fmt)

print time.time() - t

19.0049519539


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (185917440 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (185917440 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (185917440 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (185917440 pixels) exceeds limit of 89478485 pixels, 

In [81]:
display_image(im)

/oasis/projects/nsf/csd395/yuncong/Brain/vis/tmp.jpg

In [ ]:
! cd ~/csd395

In [30]:
!tar -I pigz -cf CSHL_scoremapPlusAnnotationViz.tar.gz CSHL_scoremapPlusAnnotationViz

tar: Removing leading `/' from member names
